In [1]:
import numpy as np
import os.path
from timeit import default_timer as timer
from experimentalsetup import final_pooling_evaluation
from experimentalsetup import AUPRC, AUROC
from experimentalsetup import read_tensor, read_testfilter, read_removetestlabels, read_trainfilter, read_removetrainlabels
from experimentalsetup import K_effects_A, K_drugs_A, evaluate_grid_A
from experimentalsetup import K_effects_B, K_drugs_B, evaluate_grid_B
from experimentalsetup import K_effects_C, K_drugs_C, evaluate_grid_C
from experimentalsetup import K_effects_T1, K_drugs_T1, evaluate_grid_T1
from experimentalsetup import hdv_drugs_A, hdv_effects_A
from experimentalsetup import hdv_effects_T1, hdv_effects_T1_help
from experimentalsetup import hdv_effects_B, hdv_effects_B_help
from experimentalsetup import evaluation

used ncon package


In [2]:
import matplotlib.pyplot as plt

In [3]:
import pandas as pd

In [4]:
Y = read_tensor("Data_Indexed/triples_indexed", (645, 645, 963))

In [5]:
Y.shape, type(Y[1,1,1])

((645, 645, 963), numpy.bool_)

# 1.  SETTING A: new triplets

## 1.1 Definitions and hdv vector loading

In [6]:
n_splits = 10
dirname = 'Results/SettingA_'+str(0)

In [7]:
hdv_drugs = hdv_drugs_A(dirname)
hdv_effects_a, hdv_effects_b = hdv_effects_A(Y, dirname)
hdv_effects_a+=1
hdv_effects_b+=1
hdv_drugs+=1

## 1.2 Training and saving the models 
For cross-validation, we make use of the simple structure of HDC model that is a simple sum. During training, for each test set, we bundle the data into a model M. When predicting, we construct a model that bundles all models M, and subtract the specific bundel of the test set we want to predict for. In this way, repetitive bundling of the same data can be avoided

In [8]:
for experimentnumber in range(n_splits):
    if experimentnumber < 5:
        hdv_effects = hdv_effects_a
    else:
        hdv_effects = hdv_effects_b

    canceltrain = read_removetrainlabels(dirname+'/test_'+str(experimentnumber),Y.shape).astype(bool)
    d1, d2, s = np.where(Y*canceltrain)

    Mpos = np.zeros(hdv_drugs.shape[1])
    beginexp = timer()
    size = len(d1) #100000 #
    for i in range(size): 
        Mpos+= (  hdv_drugs[d1[i],:] * hdv_drugs[d2[i],:] * hdv_effects[s[i],:] )
    Mneg = np.zeros(hdv_drugs.shape[1])
    size_ = 1*size
    d1, d2, s = np.random.choice(Y.shape[0],size_), np.random.choice(Y.shape[1],size_), np.random.choice(Y.shape[2],size_)
    for i in range(size_):
        Mneg+= (  hdv_drugs[d1[i],:] * hdv_drugs[d2[i],:] * hdv_effects[s[i],:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    np.savetxt(dirname+"/HDVresults/M_pos_"+str(experimentnumber), Mpos)
    np.savetxt(dirname+"/HDVresults/M_neg_"+str(experimentnumber), Mneg)
    

testfiler triple
time for one exp:  134.32935448100034
testfiler triple
time for one exp:  136.56229307399917
testfiler triple
time for one exp:  130.5436303639981
testfiler triple
time for one exp:  129.44243205999737
testfiler triple
time for one exp:  128.12178352900082
testfiler triple
time for one exp:  122.56653087099767
testfiler triple
time for one exp:  123.28607773200201
testfiler triple
time for one exp:  123.23185858500437
testfiler triple
time for one exp:  120.38442323899653
testfiler triple
time for one exp:  122.17843013300444


## 1.3 Pooled prediction and save
See note about cross-validation of earlier

In [9]:
models_pos = np.array([np.loadtxt(dirname+"/HDVresults/M_pos_"+str(experimentnumber)) for experimentnumber in range(n_splits)])
models = models_pos

predictions = np.nan*np.ones(Y.shape)

for experimentnumber in range(n_splits):
    if experimentnumber < 5:
        hdv_effects = hdv_effects_a
    else:
        hdv_effects = hdv_effects_b
    test = np.array(pd.read_csv(dirname+'/test_'+str(experimentnumber), ' ',header=None))
    M = np.sum(models, axis=0) - models[experimentnumber,:] 

    beginexp = timer()
    size= len(test) #200000 #
    for i in range(size):
        predictions[test[i,0], test[i,1], test[i,2]] = np.sum(M * hdv_drugs[test[i,0],:] * hdv_drugs[test[i,1],:] * hdv_effects[test[i,2],:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
#np.savetxt(dirname+"/HDVresults/res_pos", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

time for one exp:  1196.275055170001
time for one exp:  1182.0016324880053
time for one exp:  1198.6700161619956
time for one exp:  1204.0287174240002
time for one exp:  1185.6343844079965
time for one exp:  1198.3154424000022
time for one exp:  1202.6645389199984
time for one exp:  1205.3333052029993
time for one exp:  1211.0054911800034
time for one exp:  1213.145319586998
evaluaton (645, 645, 963) (645, 645, 963)
0.7676971779626663 0.731201277928467


In [10]:
models_pos = np.array([np.loadtxt(dirname+"/HDVresults/M_pos_"+str(experimentnumber)) for experimentnumber in range(n_splits)])
models_neg = np.array([np.loadtxt(dirname+"/HDVresults/M_neg_"+str(experimentnumber)) for experimentnumber in range(n_splits)])
models = models_pos - models_neg

predictions = np.nan*np.ones(Y.shape)

for experimentnumber in range(n_splits):
    if experimentnumber < 5:
        hdv_effects = hdv_effects_a
    else:
        hdv_effects = hdv_effects_b
    test = np.array(pd.read_csv(dirname+'/test_'+str(experimentnumber), ' ',header=None))
    M = np.sum(models, axis=0) - models[experimentnumber,:] 

    beginexp = timer()
    size= len(test) #200000 #
    for i in range(size):
        predictions[test[i,0], test[i,1], test[i,2]] = np.sum(M * hdv_drugs[test[i,0],:] * hdv_drugs[test[i,1],:] * hdv_effects[test[i,2],:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
np.savetxt(dirname+"/HDVresults/res_func", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

time for one exp:  1222.9988489769967
time for one exp:  1215.4451153519985
time for one exp:  1215.7975618000055
time for one exp:  1211.137882656003
time for one exp:  1190.9218416489966
time for one exp:  1190.7603842190001
time for one exp:  1206.5388407100036
time for one exp:  1199.5928634890006
time for one exp:  1200.1088273939968
time for one exp:  1191.8588472509946
evaluaton (645, 645, 963) (645, 645, 963)
0.8226324223877683 0.796500391695285


# 2. Setting T1: new drug-drug pairs

In [11]:
n_splits = 10
dirname = 'Results/SettingT1_'+str(0)

In [12]:
hdv_drugs = hdv_drugs_A('Results/SettingA_'+str(0))
hdv_drugs+=1
hdv_effects_ = hdv_effects_T1(Y, dirname, n_splits)

In [13]:
for experimentnumber in range(n_splits):
    hdv_effects = np.sign(np.sum(hdv_effects_,axis=0) - hdv_effects_[experimentnumber,:]) # do not use the representations calculated based on the test set
    hdv_effects+=1

    canceltrain = read_removetrainlabels(dirname+'/test_'+str(experimentnumber),Y.shape).astype(bool)
    d1, d2, s = np.where(Y*canceltrain)

    Mpos = np.zeros(hdv_drugs.shape[1])
    beginexp = timer()
    size =  len(d1) #100000 #
    for i in range(size): 
        Mpos+= (  hdv_drugs[d1[i],:] * hdv_drugs[d2[i],:] * hdv_effects[s[i],:] )
    Mneg = np.zeros(hdv_drugs.shape[1])
    size_ = 1*size
    d1, d2, s = np.random.choice(Y.shape[0],size_), np.random.choice(Y.shape[1],size_), np.random.choice(Y.shape[2],size_)
    for i in range(size_):
        Mneg+= (  hdv_drugs[d1[i],:] * hdv_drugs[d2[i],:] * hdv_effects[s[i],:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    np.savetxt(dirname+"/HDVresults/M_pos_"+str(experimentnumber), Mpos)
    np.savetxt(dirname+"/HDVresults/M_neg_"+str(experimentnumber), Mneg)
    

testfiler pair
time for one exp:  33.18774786299764
testfiler pair
time for one exp:  32.569002533004095
testfiler pair
time for one exp:  32.417143038001086
testfiler pair
time for one exp:  32.90326153799833
testfiler pair
time for one exp:  32.62865646699356
testfiler pair
time for one exp:  32.61964438299765
testfiler pair
time for one exp:  33.86197010699834
testfiler pair
time for one exp:  32.96563966199756
testfiler pair
time for one exp:  32.23614525599987
testfiler pair
time for one exp:  32.8676972309986


In [14]:
models_pos = np.array([np.loadtxt(dirname+"/HDVresults/M_pos_"+str(experimentnumber)) for experimentnumber in range(n_splits)])
models = models_pos

predictions = np.nan*np.ones(Y.shape)

for experimentnumber in range(n_splits):
    hdv_effects = np.sign(np.sum(hdv_effects_,axis=0) - hdv_effects_[experimentnumber,:]) # do not use the representations calculated based on the test set
    hdv_effects+=1
    
    test = np.array(pd.read_csv(dirname+'/test_'+str(experimentnumber), ' ',header=None))
    M = np.sum(models, axis=0) - models[experimentnumber,:] 

    beginexp = timer()
    size=len(test) #4000 #
    for i in range(size):
        res_ = M * hdv_drugs[test[i,0],:] * hdv_drugs[test[i,1],:]
        for j in range(Y.shape[2]):
            predictions[test[i,0], test[i,1], j] = np.sum(res_ * hdv_effects[j,:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
np.savetxt(dirname+"/HDVresults/res_pos", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

time for one exp:  286.31510815300135
time for one exp:  291.71816131800006
time for one exp:  295.55100812300225
time for one exp:  300.63555029700365
time for one exp:  299.1008428159985
time for one exp:  296.6020845890016
time for one exp:  300.69272035599715
time for one exp:  300.02548849600134
time for one exp:  286.4545087449951
time for one exp:  289.1186230309977
evaluaton (645, 645, 963) (645, 645, 963)
0.7719137937841952 0.7609546338887656


In [15]:
predictions[np.isnan(predictions)]=0
predictions+= predictions.transpose(1,0,2)

AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))

evaluaton (645, 645, 963) (645, 645, 963)
0.7722762751390467 0.7609546338887657


In [16]:
models_pos = np.array([np.loadtxt(dirname+"/HDVresults/M_pos_"+str(experimentnumber)) for experimentnumber in range(n_splits)])
models_neg = np.array([np.loadtxt(dirname+"/HDVresults/M_neg_"+str(experimentnumber)) for experimentnumber in range(n_splits)])
models = models_pos - models_neg

predictions = np.nan*np.ones(Y.shape)

for experimentnumber in range(n_splits):
    hdv_effects = np.sign(np.sum(hdv_effects_,axis=0) - hdv_effects_[experimentnumber,:]) # do not use the representations calculated based on the test set
    hdv_effects+=1
    
    test = np.array(pd.read_csv(dirname+'/test_'+str(experimentnumber), ' ',header=None))
    M = np.sum(models, axis=0) - models[experimentnumber,:] 

    beginexp = timer()
    size=len(test) #4000 #
    for i in range(size):
        res_ = M * hdv_drugs[test[i,0],:] * hdv_drugs[test[i,1],:]
        for j in range(Y.shape[2]):
            predictions[test[i,0], test[i,1], j] = np.sum(res_ * hdv_effects[j,:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
np.savetxt(dirname+"/HDVresults/res_func", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

time for one exp:  306.06686567799625
time for one exp:  296.13649689400336
time for one exp:  294.58695849399373
time for one exp:  297.79588452199823
time for one exp:  298.13720209399617
time for one exp:  298.7787527000037
time for one exp:  296.4032350799971
time for one exp:  298.6644265750001
time for one exp:  289.08434378200036
time for one exp:  287.5129440150049
evaluaton (645, 645, 963) (645, 645, 963)
0.8235109437717855 0.8078274347811651


In [17]:
predictions[np.isnan(predictions)]=0
predictions+= predictions.transpose(1,0,2)

AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))

evaluaton (645, 645, 963) (645, 645, 963)
0.8237916111596013 0.807827434781165


# 3. SETTING B = (False, True, True): new drugs

In [18]:
def testdrugs_(i,n_drugs, n_splits):
    dx = int(n_drugs/n_splits+0.5)
    a,b = i*dx , min((i+1)*dx, n_drugs)
    return a,b

In [19]:
n_splits = 10
dirname = 'Results/SettingB_'+str(0)

In [ ]:
hdv_drugs = hdv_drugs_A('Results/SettingA_'+str(0))
hdv_drugs+=1
hdv_effects_ = hdv_effects_B(Y, dirname, n_splits)

In [ ]:
for split1 in range(n_splits):
    testdrugs1 = np.arange(testdrugs_(split1,len(Y), n_splits)[0],testdrugs_(split1,len(Y),n_splits)[1])
    for split2 in range(n_splits):
        trainfolds1 = np.delete(np.array(range(n_splits)), split1)
        trainfolds2 = np.delete(np.array(range(n_splits)), split2)
        hdv_effects = np.sign(np.sum(hdv_effects_[trainfolds1,:,:,:][:,trainfolds1,:,:], axis=(0,1))).astype('float64')
        hdv_effects+=1
        
        testdrugs2 = np.arange(testdrugs_(split2,len(Y), n_splits)[0],testdrugs_(split2,len(Y),n_splits)[1])        
        Ytest = np.full(Y.shape, False)
        Ytest[testdrugs1[0]:testdrugs1[-1]+1, testdrugs2[0]:testdrugs2[-1]+1,:] = Y[testdrugs1[0]:testdrugs1[-1]+1, testdrugs2[0]:testdrugs2[-1]+1,:]
        
        d1, d2, s = np.where(Ytest)

        
        Mpos = np.zeros(hdv_drugs.shape[1])
        beginexp = timer()
        size =  len(d1) #100000 #
        for i in range(size): 
            Mpos+= (  hdv_drugs[d1[i],:] * hdv_drugs[d2[i],:] * hdv_effects[s[i],:] )
        Mneg = np.zeros(hdv_drugs.shape[1])
        size_ = 1*size
        d1, d2, s = np.random.choice(Y.shape[0],size_), np.random.choice(Y.shape[1],size_), np.random.choice(Y.shape[2],size_)
        for i in range(size_):
            Mneg+= (  hdv_drugs[d1[i],:] * hdv_drugs[d2[i],:] * hdv_effects[s[i],:] )
        endexp = timer()
        print("time for one exp: " , (endexp-beginexp))
        np.savetxt(dirname+"/HDVresults/M_pos_"+str(split1)+"_"+str(split2), Mpos)
        np.savetxt(dirname+"/HDVresults/M_neg_"+str(split1)+"_"+str(split2), Mneg)


In [ ]:
models_pos = np.array([[np.loadtxt(dirname+"/HDVresults/M_pos_"+str(split1)+"_"+str(split2)) for split1 in range(n_splits)] for split2 in range(n_splits)])
models = models_pos

predictions = np.nan*np.ones(Y.shape)

for split in range(n_splits):
    testdrugs = np.arange(testdrugs_(split,len(Y), n_splits)[0],testdrugs_(split,len(Y),n_splits)[1])
    traindrugs = np.delete(np.array(range(645)),testdrugs)
    trainfolds = np.delete(np.array(range(n_splits)), split1)
    hdv_effects = np.sign(np.sum(hdv_effects_[trainfolds,:,:,:][:,trainfolds,:,:], axis=(0,1))).astype('float64')
    hdv_effects+=1
        
    M = np.sum(models[trainfolds,:,:][:,trainfolds,:], axis=(0,1))

    beginexp = timer()
    for i in testdrugs:
        res_ = M * hdv_drugs[i,:]
        for j in traindrugs:
            res__ = res_ * hdv_drugs[j,:]
            for k in range(Y.shape[2]):
                predictions[i,j,k] = np.sum(res__ * hdv_effects[k,:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
    
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
np.savetxt(dirname+"/HDVresults/res_pos", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

In [ ]:
models_pos = np.array([[np.loadtxt(dirname+"/HDVresults/M_pos_"+str(split1)+"_"+str(split2)) for split1 in range(n_splits)] for split2 in range(n_splits)])
models_neg = np.array([[np.loadtxt(dirname+"/HDVresults/M_neg_"+str(split1)+"_"+str(split2)) for split1 in range(n_splits)] for split2 in range(n_splits)])
models = models_pos - models_neg

predictions = np.nan*np.ones(Y.shape)

for split in range(n_splits):
    testdrugs = np.arange(testdrugs_(split,len(Y), n_splits)[0],testdrugs_(split,len(Y),n_splits)[1])
    traindrugs = np.delete(np.array(range(645)),testdrugs)
    trainfolds = np.delete(np.array(range(n_splits)), split1)
    hdv_effects = np.sign(np.sum(hdv_effects_[trainfolds,:,:,:][:,trainfolds,:,:], axis=(0,1))).astype('float64')
    hdv_effects+=1
        
    M = np.sum(models[trainfolds,:,:][:,trainfolds,:], axis=(0,1))

    beginexp = timer()
    for i in testdrugs:
        res_ = M * hdv_drugs[i,:]
        for j in traindrugs:
            res__ = res_ * hdv_drugs[j,:]
            for k in range(Y.shape[2]):
                predictions[i,j,k] = np.sum(res__ * hdv_effects[k,:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
    
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
np.savetxt(dirname+"/HDVresults/res_func", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

# 4. Setting C: two new drugs
the features and training will be exactly the same as in setting B. Only the test set is different: where B's test set is made of the combinations of unseen drugs and seen drugs, C's test set is the combination of unseen drugs and unseen drugs. The train sets are in both cases the combinations of seen drugs and seen drugs

In [ ]:
n_splits = 10
dirname = 'Results/SettingC_'+str(0) # this will be used to save the evaluation accuracy
dirname_B = 'Results/SettingB_'+str(0) #this will be used to load the features and the trained models

In [ ]:
hdv_drugs = hdv_drugs_A('Results/SettingA_'+str(0))
hdv_drugs+=1
hdv_effects_ = hdv_effects_B(Y, dirname_B, n_splits)

In [ ]:
models_pos = np.array([[np.loadtxt(dirname_B+"/HDVresults/M_pos_"+str(split1)+"_"+str(split2)) for split1 in range(n_splits)] for split2 in range(n_splits)])
models = models_pos

predictions = np.nan*np.ones(Y.shape)

for split in range(n_splits):
    testdrugs = np.arange(testdrugs_(split,len(Y), n_splits)[0],testdrugs_(split,len(Y),n_splits)[1])
    trainfolds = np.delete(np.array(range(n_splits)), split1)
    hdv_effects = np.sign(np.sum(hdv_effects_[trainfolds,:,:,:][:,trainfolds,:,:], axis=(0,1))).astype('float64')
    hdv_effects+=1
        
    M = np.sum(models[trainfolds,:,:][:,trainfolds,:], axis=(0,1))

    beginexp = timer()
    for i in testdrugs:
        res_ = M * hdv_drugs[i,:]
        for j in testdrugs:
            res__ = res_ * hdv_drugs[j,:]
            for k in range(Y.shape[2]):
                predictions[i,j,k] = np.sum(res__ * hdv_effects[k,:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
    
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
np.savetxt(dirname+"/HDVresults/res_pos", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

In [ ]:
models_pos = np.array([[np.loadtxt(dirname_B+"/HDVresults/M_pos_"+str(split1)+"_"+str(split2)) for split1 in range(n_splits)] for split2 in range(n_splits)])
models_neg = np.array([[np.loadtxt(dirname_B+"/HDVresults/M_neg_"+str(split1)+"_"+str(split2)) for split1 in range(n_splits)] for split2 in range(n_splits)])
models = models_pos - models_neg

predictions = np.nan*np.ones(Y.shape)

for split in range(n_splits):
    testdrugs = np.arange(testdrugs_(split,len(Y), n_splits)[0],testdrugs_(split,len(Y),n_splits)[1])
    trainfolds = np.delete(np.array(range(n_splits)), split)
    hdv_effects = np.sign(np.sum(hdv_effects_[trainfolds,:,:,:][:,trainfolds,:,:], axis=(0,1))).astype('float64')
    hdv_effects+=1
        
    M = np.sum(models[trainfolds,:,:][:,trainfolds,:], axis=(0,1))

    beginexp = timer()
    for i in testdrugs:
        res_ = M * hdv_drugs[i,:]
        for j in testdrugs:
            res__ = res_ * hdv_drugs[j,:]
            for k in range(Y.shape[2]):
                predictions[i,j,k] = np.sum(res__ * hdv_effects[k,:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
    
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
np.savetxt(dirname+"/HDVresults/res_func", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

In [ ]:
models_pos = np.array([np.loadtxt(dirname+"/HDVresults/M_pos_"+str(experimentnumber)) for experimentnumber in range(n_splits)])
models = models_pos

predictions = np.nan*np.ones(Y.shape)

for split in range(n_splits):
    testdrugs = np.arange(testdrugs_(split,len(Y), n_splits)[0],testdrugs_(split,len(Y),n_splits)[1])
    trainfolds = np.delete(np.array(range(n_splits)), split1)
    hdv_effects = np.sign(np.sum(hdv_effects_[trainfolds,:,:,:][:,trainfolds,:,:], axis=(0,1))).astype('float64')
    hdv_effects+=1
        
    M = np.sum(models_pos[trainfolds,:,:][:,trainfolds,:], axis=(0,1))

    test = np.array(pd.read_csv(dirname+'/test_'+str(experimentnumber), ' ',header=None))
    M = np.sum(models, axis=0) - models[experimentnumber,:] 

    beginexp = timer()
    size=2 #len(test) #
    for i in testdrugs[:size]:
        res_ = M * hdv_drugs[i,:] #* hdv_drugs[test[i,1],:]
        for j in testdrugs:
            res__ = res_ * hdv_drugs[j,:]
            for k in range(Y.shape[2]):
                predictions[i,j,k] = np.sum(res__ * hdv_effects[k,:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
    
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
np.savetxt(dirname+"/HDVresults/res_pos", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

In [ ]:
models_pos = np.array([np.loadtxt(dirname+"/HDVresults/M_pos_"+str(experimentnumber)) for experimentnumber in range(n_splits)])
models = models_pos


predictions = np.nan*np.ones(Y.shape)

for experimentnumber in range(n_splits):
    if experimentnumber < 5:
        hdv_effects = hdv_effects_a
    else:
        hdv_effects = hdv_effects_b
    testdrugs = np.arange(testdrugs_(experimentnumber,len(Y), n_splits)[0],testdrugs_(experimentnumber,len(Y),n_splits)[1])
    M = np.sum(models, axis=(0,1)) - np.sum(models,axis=0)[experimentnumber,:] - np.sum(models,axis=0)[experimentnumber,:] + models[experimentnumber,experimentnumber,:]

    beginexp = timer()
    size=2 #len(test) #
    for i in testdrugs[:size]:
        res_ = M * hdv_drugs[i,:] #* hdv_drugs[test[i,1],:]
        for j in testdrugs:
            res__ = res_ * hdv_drugs[j,:]
            for k in range(Y.shape[2]):
                predictions[i,j,k] = np.sum(res__ * hdv_effects[k,:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
np.save(dirname+"/HDVresults/predictions_pos", predictions)


In [21]:
hdv_drugs = hdv_drugs_A('Results/SettingA_'+str(0))
hdv_effects_a, hdv_effects_b = hdv_effects_A(Y, 'Results/SettingA_'+str(0))
hdv_effects_a+=1
hdv_effects_b+=1
hdv_drugs+=1

In [23]:
for experimentnumber in range(n_splits):
    if experimentnumber < 5:
        hdv_effects = hdv_effects_a
    else:
        hdv_effects = hdv_effects_b

    testdrugs = np.arange(testdrugs_(experimentnumber,len(Y), n_splits)[0],testdrugs_(experimentnumber,len(Y),n_splits)[1])
    Ytest = np.full(Y.shape, False)
    Ytest[testdrugs, :, :]  = Y[testdrugs, :, :]
    d1, d2, s = np.where(Ytest)

    M = np.zeros(hdv_drugs.shape[1])
    beginexp = timer()
    size = len(d1) #100000 #
    for i in range(size): 
        M+= (  hdv_drugs[d1[i],:] * hdv_drugs[d2[i],:] * hdv_effects[s[i],:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    np.savetxt(dirname+"/HDVresults/M_unnormalized_"+str(experimentnumber), M)

time for one exp:  70.96228308100035
time for one exp:  67.06033425100031
time for one exp:  50.110947727000166
time for one exp:  63.75925853800072
time for one exp:  59.480018497000856
time for one exp:  67.33223096399888
time for one exp:  56.59757191999961
time for one exp:  55.921421679999185
time for one exp:  48.26102440700015
time for one exp:  58.35788145600054


In [24]:

models_pos = np.array([np.loadtxt(dirname+"/HDVresults/M_unnormalized_"+str(experimentnumber)) for experimentnumber in range(n_splits)])
models = models_pos

predictions = np.nan*np.ones(Y.shape)

for experimentnumber in range(n_splits):
    if experimentnumber < 5:
        hdv_effects = hdv_effects_a
    else:
        hdv_effects = hdv_effects_b
    testdrugs = np.arange(testdrugs_(experimentnumber,len(Y), n_splits)[0],testdrugs_(experimentnumber,len(Y),n_splits)[1])
    M = np.sum(models, axis=0) - models[experimentnumber,:] 

    beginexp = timer()
    size=2 #len(test) #
    for i in testdrugs[:size]:
        res_ = M * hdv_drugs[i,:] #* hdv_drugs[test[i,1],:]
        for j in range(Y.shape[1]):
            res__ = res_ * hdv_drugs[j,:]
            for k in range(Y.shape[2]):
                predictions[i,j,k] = np.sum(res__ * hdv_effects[k,:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
np.save(dirname+"/HDVresults/predictions_pos", predictions)


time for one exp:  47.89425516099982
time for one exp:  43.01237861399932
time for one exp:  42.88106956199954
time for one exp:  41.70123533900005
time for one exp:  42.11873250999997
time for one exp:  43.8490694459997
time for one exp:  43.62824070199895
time for one exp:  44.018001324999204
time for one exp:  44.58485619900057
time for one exp:  43.30023003600036


In [25]:
predictions = np.load(dirname+"/HDVresults/predictions_pos.npy")
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
np.savetxt(dirname+"/HDVresults/res_pos", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

evaluaton (645, 645, 963) (645, 645, 963)
0.7437180398466569 0.6300519288993759


In [ ]:
models_pos = np.array([np.loadtxt(dirname+"/HDVresults/M_pos_"+str(experimentnumber)) for experimentnumber in range(n_splits)])
models_neg = np.array([np.loadtxt(dirname+"/HDVresults/M_neg_"+str(experimentnumber)) for experimentnumber in range(n_splits)])
models = models_pos - models_neg

predictions = np.nan*np.ones(Y.shape)

for experimentnumber in range(n_splits):
    if experimentnumber < 5:
        hdv_effects = hdv_effects_a
    else:
        hdv_effects = hdv_effects_b
    test = np.array(pd.read_csv(dirname+'/test_'+str(experimentnumber), ' ',header=None))
    M = np.sum(models, axis=0) - models[experimentnumber,:] 

    beginexp = timer()
    size=len(test) #100000 # 
    for i in range(size):
        predictions[test[i,0], test[i,1], test[i,2]] = np.sum(M * hdv_drugs[test[i,0],:] * hdv_drugs[test[i,1],:] * hdv_effects[test[i,2],:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
np.savetxt(dirname+"/HDVresults/res_func", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

# 4. SETTING C = (False, False, True): two new drugs

In [7]:
n_splits = 10
dirname = 'Results/SettingC_'+str(0)

In [8]:
hdv_drugs = hdv_drugs_A('Results/SettingA_'+str(0))
hdv_effects_a, hdv_effects_b = hdv_effects_A(Y, 'Results/SettingA_'+str(0))
hdv_effects_a+=1
hdv_effects_b+=1
hdv_drugs+=1

In [76]:
for experimentnumber in range(n_splits):
    for experimentnumber_ in range(n_splits):
        if experimentnumber < 5:
            hdv_effects = hdv_effects_a
        else:
            hdv_effects = hdv_effects_b

        testdrugs = np.arange(testdrugs_(experimentnumber,len(Y), n_splits)[0],testdrugs_(experimentnumber,len(Y),n_splits)[1])
        testdrugs__ = np.arange(testdrugs_(experimentnumber_,len(Y), n_splits)[0],testdrugs_(experimentnumber_,len(Y),n_splits)[1])
        
        Ytest = np.full(Y.shape, False)
        #Ytest[testdrugs, :,:][:,testdrugs__, :] = Y[testdrugs, :,:][:,testdrugs__, :]
        Ytest[testdrugs[0]:testdrugs[-1]+1, testdrugs__[0]:testdrugs__[-1]+1,:] = Y[testdrugs[0]:testdrugs[-1]+1, testdrugs__[0]:testdrugs__[-1]+1,:]
        
        d1, d2, s = np.where(Ytest)

        M = np.zeros(hdv_drugs.shape[1])
        beginexp = timer()
        size = len(d1) #100000 #
        for i in range(size): 
            M+= (  hdv_drugs[d1[i],:] * hdv_drugs[d2[i],:] * hdv_effects[s[i],:] )
        #size_ = 1*size
        #d1, d2, s = np.random.choice(Y.shape[0],size_), np.random.choice(Y.shape[1],size_), np.random.choice(Y.shape[2],size_)
        #for i in range(size_): 
        #    M-= (  hdv_drugs[d1[i],:] * hdv_drugs[d2[i],:] * hdv_effects[s[i],:] )
        endexp = timer()
        print("time for one exp: " , (endexp-beginexp))
        np.savetxt(dirname+"/HDVresults/M_unnormalized_"+str(experimentnumber)+"_"+str(experimentnumber_), M)

time for one exp:  7.463480355003412
time for one exp:  6.9878928979997
time for one exp:  5.5997927250027715
time for one exp:  6.081257435002044
time for one exp:  5.6753469030009
time for one exp:  7.109685886000079
time for one exp:  6.3300237500006915
time for one exp:  6.150484365000011
time for one exp:  5.404742991002422
time for one exp:  6.119205925002461
time for one exp:  6.789032870001392
time for one exp:  5.899276070998894
time for one exp:  4.69871392999994
time for one exp:  5.778850953000074
time for one exp:  5.495821954002167
time for one exp:  7.452004907998344
time for one exp:  6.41458115000205
time for one exp:  6.324204717999237
time for one exp:  5.259655500001827
time for one exp:  6.2599810769970645
time for one exp:  5.649337792001461
time for one exp:  5.10864550900078
time for one exp:  3.858914819000347
time for one exp:  4.386332836998918
time for one exp:  4.2059951559967885
time for one exp:  5.591531512000074
time for one exp:  4.492616890001955
time

In [79]:

models_pos = np.array([[np.loadtxt(dirname+"/HDVresults/M_unnormalized_"+str(experimentnumber)+"_"+str(experimentnumber_)) for experimentnumber in range(n_splits)] for experimentnumber_ in range(n_splits)])
models = models_pos

predictions = np.nan*np.ones(Y.shape)

for experimentnumber in range(n_splits):
    if experimentnumber < 5:
        hdv_effects = hdv_effects_a
    else:
        hdv_effects = hdv_effects_b
    testdrugs = np.arange(testdrugs_(experimentnumber,len(Y), n_splits)[0],testdrugs_(experimentnumber,len(Y),n_splits)[1])
    M = np.sum(models, axis=(0,1)) - np.sum(models,axis=0)[experimentnumber,:] - np.sum(models,axis=0)[experimentnumber,:] + models[experimentnumber,experimentnumber,:]

    beginexp = timer()
    size=2 #len(test) #
    for i in testdrugs[:size]:
        res_ = M * hdv_drugs[i,:] #* hdv_drugs[test[i,1],:]
        for j in testdrugs:
            res__ = res_ * hdv_drugs[j,:]
            for k in range(Y.shape[2]):
                predictions[i,j,k] = np.sum(res__ * hdv_effects[k,:] )
    endexp = timer()
    print("time for one exp: " , (endexp-beginexp))
    
np.save(dirname+"/HDVresults/predictions_pos", predictions)


time for one exp:  4.014834584999335
time for one exp:  3.9915483459990355
time for one exp:  4.003380614001799
time for one exp:  3.9843636190016696
time for one exp:  4.017887336001877
time for one exp:  4.044881303998409
time for one exp:  4.052052523999009
time for one exp:  4.323146897000697
time for one exp:  4.047937997002009
time for one exp:  3.7434131590016477


In [80]:
predictions = np.load(dirname+"/HDVresults/predictions_pos.npy")
AUROC_dd = evaluation(predictions, Y, AUROC, (False, False, True))
AUROC_e = evaluation(predictions, Y, AUROC, (True, True, False))
print(np.nanmean(AUROC_dd), np.nanmean(AUROC_e))
np.savetxt(dirname+"/HDVresults/res_pos", [np.nanmean(AUROC_dd), np.nanmean(AUROC_e)])

evaluaton (645, 645, 963) (645, 645, 963)
0.7327513102305624 0.7019753795163216


# Recalculate final predictions and save

In [ ]:
import pandas as pd
from threestep import NstepRegressor, KroneckerKernel
from experimentalsetup import testdrugs_
def final_pooling_prediction(Y,setting,dirname,n_splits,metric,scheme):
    Yhat = np.zeros(Y.shape)
    if setting == (False,True,True) or setting == (False,False,True): # in these settings there is not in every fold an estimation, so we cannot simply add predictions, non predicted must remain nan
        Yhat[:] = np.nan
    for experimentnumber in range(n_splits):
        opi = str(scheme)+metric.__name__
        optimizationresults = pd.read_csv(dirname+'/hypamoptimization_'+str(experimentnumber), sep='\t')#, names=colnames)
        o = list(optimizationresults[optimizationresults[opi] == max(optimizationresults[opi])].iloc[0,0:3])
        nstep = NstepRegressor(o)
        print(o)
        if setting == (True, True, True) or setting == 'T1':

            K1 = K2 =  np.loadtxt(dirname+"/Kdrugs_rbf")
            K3 = np.loadtxt(dirname+"/Keffects_cos_"+str(experimentnumber))
            K = KroneckerKernel([K1, K2, K3])

            #load the matrices into Ktrain kroneckerkernel
            canceltest = read_removetestlabels(dirname+'/test_'+str(experimentnumber),Y.shape)
            canceltrain = read_removetrainlabels(dirname+'/test_'+str(experimentnumber),Y.shape)
            Yest = nstep.fit_predict_LO(K,Y*canceltest, setting)
            Yimputed = Y + (Yest-Y)*canceltrain #for the test data, Y is replaced by an estimation. This improves quality of training data
            Yest = nstep.fit_predict_LO(K,Yimputed,setting)
            Yhat = Yhat + Yest*canceltrain # put training estimations to zero and add these to the Yhat, if this is done for every fold, the Yhat tensor is a pooled estimation from the folds

        if setting == (False, True, True):
            testdrugs = np.arange(testdrugs_(experimentnumber,len(Y),n_splits)[0],testdrugs_(experimentnumber,len(Y), n_splits)[1])
            Ytrain = np.delete(np.delete(Y, testdrugs, axis=0), testdrugs, axis=1)
            Ytest = np.delete(Y, testdrugs, axis=1)[testdrugs[0]:testdrugs[-1], :,:]
            K1 = K2 =  np.loadtxt(dirname+"/Kdrugs_rbf")
            K3 = np.loadtxt(dirname+"/Keffects_cos_"+str(experimentnumber))
            Ktrain = KroneckerKernel([np.delete(np.delete(K1, testdrugs, axis=0), testdrugs,axis=1), np.delete(np.delete(K2, testdrugs, axis=0), testdrugs,axis=1), K3])
            Ktest = KroneckerKernel([np.delete(K1, testdrugs, axis=1)[testdrugs[0]:testdrugs[-1], :], np.delete(np.delete(K2, testdrugs, axis=0), testdrugs,axis=1), K3])
            nstep.fit(Ktrain, Ytrain.astype('float64'))
            Yest = nstep.predict(Ktest) #.astype('float32')
            # assign the predictions correctly: note we did not, due to symmetry estimate for a diagonal block (these acutally coresspond to setting C = (False,False,True)). This is why there is a upper part assignment and lower part assignment (below the diagonal block)
            print(Yhat.shape, Ytrain.shape, Ytest.shape, Yest.shape,len(testdrugs))

            Yhat[testdrugs[0]:testdrugs[-1], :testdrugs[0] , :] = Yest[:,:testdrugs[0],:]
            Yhat[testdrugs[0]:testdrugs[-1], testdrugs[-1]+1: , :] = Yest[:,testdrugs[0]:,:]

        if setting == (False, False, True):
            testdrugs = np.arange(testdrugs_(experimentnumber,len(Y),n_splits)[0],testdrugs_(experimentnumber,len(Y), n_splits)[1])
            Ytrain = np.delete(np.delete(Y, testdrugs, axis=0), testdrugs, axis=1)
            Ytest = Y[testdrugs[0]:testdrugs[-1], testdrugs[0]:testdrugs[-1],:]
            K1 = K2 =  np.loadtxt(dirname+"/Kdrugs_rbf")
            K3 = np.loadtxt(dirname+"/Keffects_cos_"+str(experimentnumber))
            Ktrain = KroneckerKernel([np.delete(np.delete(K1, testdrugs, axis=0), testdrugs,axis=1), np.delete(np.delete(K2, testdrugs, axis=0), testdrugs,axis=1), K3])
            Ktest = KroneckerKernel([np.delete(K1, testdrugs, axis=1)[testdrugs[0]:testdrugs[-1], :], np.delete(K2, testdrugs, axis=1)[testdrugs[0]:testdrugs[-1], :], K3])

            nstep.fit(Ktrain, Ytrain.astype('float64'))
            Yest = nstep.predict(Ktest) #.astype('float32')
            # assign the predictions correctly: here we assign the diagonal blocks. Note that here we need to set the diagonal elements to nan to neglegt in the evaluation.
            Yhat[testdrugs[0]:testdrugs[-1], testdrugs[0]:testdrugs[-1] , :] = Yest
            ndiag = len(Yhat)
            for i in range(ndiag):
                Yhat[i,i,:] = np.nan



        print(experimentnumber)
    np.save('final_predictions_'+str(setting)+ metric.__name__+str(scheme),Yhat)




In [ ]:
n_splits = 10
dirname = 'Results/SettingA_0'

metrics = [AUROC, AUPRC]
schemes = [(True, True, False), (False, False, True)]

for metric in metrics:
    for scheme in schemes:
        final_pooling_prediction(Y,(True, True, True),dirname,n_splits,metric,scheme)

In [ ]:
n_splits = 10
dirname = 'Results/SettingT1_0'

metrics = [AUROC, AUPRC]
schemes = [(True, True, False), (False, False, True)]

for metric in metrics:
    for scheme in schemes:
        final_pooling_prediction(Y,'T1',dirname,n_splits,metric,scheme)

In [ ]:
n_splits = 10
dirname = 'Results/SettingB_0'

metrics = [AUROC, AUPRC]
schemes = [(True, True, False), (False, False, True)]

for metric in metrics:
    for scheme in schemes:
        final_pooling_prediction(Y,(False, True, True),dirname,n_splits,metric,scheme)

In [ ]:
n_splits = 10
dirname = 'Results/SettingC_0'

metrics = [AUROC, AUPRC]
schemes = [(True, True, False), (False, False, True)]

for metric in metrics:
    for scheme in schemes:
        final_pooling_prediction(Y,(False, False, True),dirname,n_splits,metric,scheme)